In [59]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import re
import spacy
from transformers import pipeline
import seaborn as sns
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from torch import cuda
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import BCEWithLogitsLoss
import nltk
from transformers import Trainer
nltk.download('words')
from nltk.corpus import words
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
from collections import Counter
import torch.optim as optim
from torch.optim import lr_scheduler
import os
from nltk.stem.snowball import SnowballStemmer
import spacy
from tqdm import tqdm


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df_train = pd.read_csv('training_preprocesat.csv')

In [4]:
len(df_train)

70575

In [5]:
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
robert.resize_token_embeddings(len(tokenizer))

Embedding(50005, 768)

In [6]:
df_shuffled = df_train.sample(frac=0.2, random_state=1000)
train_ratio = 0.8
train_size = int(train_ratio * len(df_shuffled))
df_train = df_shuffled[:train_size]
df_val = df_shuffled[train_size:]

In [130]:
X_train = []
y_train = []
aux = list(zip(df_train['title'].tolist(), df_train['class'].tolist()))

In [131]:
for title, label in tqdm(aux):
    input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
    outputs = robert(input_ids)
    mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
    X_train.append(mean_pooling)
    y_train.append(label)

100%|██████████| 11292/11292 [06:34<00:00, 28.59it/s]


In [135]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [136]:
X_val = []
y_val = []
aux = list(zip(df_val['title'].tolist(), df_val['class'].tolist()))

In [165]:
np.save('x_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('x_val.npy', X_val)
np.save('y_val.npy', y_val)

In [137]:
for title, label in tqdm(aux):
    input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
    outputs = robert(input_ids)
    mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
    X_val.append(mean_pooling)
    y_val.append(label)

100%|██████████| 2823/2823 [01:31<00:00, 30.78it/s]


In [138]:
X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
# AICI INCEPE MODDELUL PENTRU TITLU

In [17]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index]), torch.tensor(self.y[index])

In [18]:
class ClssificationNetwork(nn.Module):
    def __init__(self):
        super(ClssificationNetwork, self).__init__()
        self.linear = nn.Linear(768, 128)
        self.classifier = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.classifier(x)
        x = self.sigmoid(x)
        
        return x

In [19]:
train_set = CustomDataset(X_train, y_train)
val_set = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_set, shuffle=True, batch_size=16)
val_loader = DataLoader(val_set, shuffle=True, batch_size=16)

NameError: name 'X_val' is not defined

In [25]:
def get_training_data():
    first = pd.read_csv('first.csv')
    first_titles = np.load('training_splits/firsttitle.npy')
    first_labels = first['class'].tolist()

    second = pd.read_csv('second.csv')
    second_titles = np.load('training_splits/secondtitle.npy')
    second_labels = second['class'].tolist()

    third = pd.read_csv('third.csv')
    third_titles = np.load('training_splits/thirdtitle.npy')
    third_labels = third['class'].tolist()

    # fourth = pd.read_csv('fourth.csv')
    # fourth_titles = np.load('training_splits/fourthtitle.npy')
    # fourth_labels = fourth['class'].tolist()

    # all_titles = np.concatenate((first_titles, second_titles, third_titles, fourth_titles), axis=0)
    # all_labels = np.concatenate((first_labels, second_labels, third_labels, fourth_labels))

    all_titles = np.concatenate((first_titles, second_titles, third_titles), axis=0)
    all_labels = np.concatenate((first_labels, second_labels, third_labels))
    
    return all_titles, all_labels

def get_validation_data():
    fourth = pd.read_csv('fourth.csv')
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_labels = fourth['class'].tolist()
    
    return fourth_titles, fourth_labels

In [26]:
X_train, y_train = get_training_data()
X_val, y_val = get_validation_data()

In [27]:
train_set = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_set, shuffle=True, batch_size=16)

val_set = CustomDataset(X_val, y_val)
val_loader = DataLoader(val_set, shuffle=True, batch_size=16)

In [31]:
model_titles = ClssificationNetwork().to(device)
optimizer = optim.Adam(model_titles.parameters(),lr=3e-5)
criterion = nn.BCELoss(weight=torch.tensor([len(y_train) / np.sum(y_train)]).to(device))

In [32]:
EPOCHS = 10

In [33]:
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

for epoch in range(EPOCHS):
    model_titles.train()
    running_loss = 0
    running_acc = 0

    train_pred_labels = []
    train_true_labels = []

    for X, y in tqdm(train_loader):
        y = y.to(device, dtype=torch.long)
        X = X.to(device, dtype=torch.float)

        output = model_titles(X)

        output.squeeze(1)
        pred = output > 0.5

        train_pred_labels.extend(pred.cpu().numpy().tolist())
        train_true_labels.extend(y.cpu().numpy().tolist())
        
        loss = criterion(output.squeeze(1), y.float())
        running_loss += loss.item()

        model_titles.zero_grad()
        loss.backward()
        optimizer.step()

    pred_labels = []
    true_labels = []

    model_titles.eval()
    val_loss = 0
    with torch.no_grad():
        for X, y in tqdm(val_loader):
            y = y.to(device, dtype=torch.long)
            X = X.to(device, dtype=torch.float)

            output = model_titles(X)
            val_loss += criterion(output.squeeze(1), y.float()).item()
            output.squeeze(1)

            pred = (output > 0.5)
            pred_labels.extend(pred.cpu().numpy().tolist())
            true_labels.extend(y.cpu().numpy().tolist())
    # 
    # model_titles.train()
    # 
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
    precision_train, recall_train, f1_train, _ = precision_recall_fscore_support(train_pred_labels, train_true_labels, average='macro')

    print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train}")
    print(f"Epoch: {epoch + 1} | Val Loss: {val_loss / len(val_set)} | Val Precision: {precision} | Val Recall: {recall} | Val F1: {f1}")

torch.save(model_titles.state_dict(), 'models/titleClassifier.pth')

  0%|          | 0/3309 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 1103/1103 [00:00<00:00, 1833.03it/s]


Epoch: 1 | Train Loss 0.02702945787560611  | Train Precision: 0.8664571513594419 | Train Recall: 0.8916779125021234 | Train F1: 0.8765778707217409
Epoch: 1 | Val Loss: 0.018586312897491974 | Val Precision: 0.9219590068510951 | Val Recall: 0.9150444870558101 | Val F1: 0.918326416692937


  0%|          | 0/3309 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 1103/1103 [00:00<00:00, 1765.21it/s]


Epoch: 2 | Train Loss 0.017729077524044002  | Train Precision: 0.9182815721066668 | Train Recall: 0.9260642146383258 | Train F1: 0.9219269877684528
Epoch: 2 | Val Loss: 0.016414185522874488 | Val Precision: 0.9335332887292072 | Val Recall: 0.9207401817121437 | Val F1: 0.9265843998072121


  0%|          | 0/3309 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 1103/1103 [00:00<00:00, 1716.78it/s]


...

Epoch: 8 | Train Loss 0.013766489872183967  | Train Precision: 0.9394188191243996 | Train Recall: 0.9433467045133672 | Train F1: 0.9413212196971346
Epoch: 8 | Val Loss: 0.0133785297907815 | Val Precision: 0.9428173912796486 | Val Recall: 0.9398735023674043 | Val F1: 0.9413147423824897


  0%|          | 0/3309 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 1103/1103 [00:00<00:00, 1590.84it/s]


Epoch: 9 | Train Loss 0.013484668420497808  | Train Precision: 0.940070442383627 | Train Recall: 0.9439720378334209 | Train F1: 0.9419606240449045
Epoch: 9 | Val Loss: 0.013166058367185913 | Val Precision: 0.9495208180432475 | Val Recall: 0.9382952304582982 | Val F1: 0.9434984369715675


  0%|          | 0/3309 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 1103/1103 [00:00<00:00, 2065.11it/s]


Epoch: 10 | Train Loss 0.013219047383220144  | Train Precision: 0.9417740673657766 | Train Recall: 0.945798297095978 | Train F1: 0.9437221482262386
Epoch: 10 | Val Loss: 0.012870887587749501 | Val Precision: 0.9443051435251357 | Val Recall: 0.9412489556813894 | Val F1: 0.9427441467613633


In [34]:
# GATA MODELULU TITLU
# GATA MODELULU TITLU

In [75]:
def test_embedding():
    df = pd.read_csv('test_preprocesat.csv')
    X = []
    
    for title in tqdm(df['title']):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X.append(mean_pooling)
        
    X = np.array(X)
    np.save('test_embeddings.npy', X)
    return X

In [76]:
# X_test = test_embedding()
X_test = np.load('test_embeddings.npy')

In [2]:
X_test = np.load('test_embeddings.npy')

In [6]:
model = ClssificationNetwork().to(device)
model.load_state_dict(torch.load('haisavedem.pth'))

<All keys matched successfully>

In [77]:
class CustomTestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index]), y

In [78]:
model.eval()
predictions = []
indices = []
cnt = 0
with torch.no_grad():
    for i in tqdm(range(len(X_test))):
        axu = torch.tensor(X_test[i], dtype=torch.float).to(device)
        prediction = model(axu) > 0.5
        predictions.append(prediction.item())
        indices.append(cnt)
        cnt += 1
model.train()

100%|██████████| 36669/36669 [00:08<00:00, 4227.45it/s]


ClssificationNetwork(
  (linear): Linear(in_features=768, out_features=128, bias=True)
  (classifier): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [227]:
len(ids)

36669

In [79]:
submisie = pd.DataFrame({'id':indices, 'class':list(map(int, predictions))})
submisie.to_csv('submission.csv', index=False)

In [35]:
# preprocess the data for the model
def replace_unwanted_characters(df):
    df['content'] = df['content'].fillna('')
    df['title'] = df['title'].fillna('')
    df['title'] = df['title'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    df['content'] = df['content'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    return df

repeated_characters = [x * 3 for x in "AĂÂBCDEFGHÎJKLMNOPQRSȘTȚUVWXYZaăâbcdefghîjklmnopqrsștțuvwxyz,.-/';[]-!@#$%^&*()?+"]
repeated_characters.append('iiii')
repeated_characters.append('IIII')

profanity_list = [
    "muie",
    "laba",
    "labă",
    "pula",
    "pulă",
    "pizda",
    "pizdă",
    "ce pula",
    "ce pulă",
    "ce pizda",
    "ce pizdă",
    "caca",
    "cacat",
    "căcat",
    "pipi",
    "pisat",
    "pișat",
    "pishat",
    "rahat",
    "kkt",
    "kk",
    "plm",
    "ma fut",
    "mă fut",
    "ma cac",
    "mă cac",
    "ma pis",
    "mă pis",
    "ma pish",
    "mă pish",
    "pwla",
    "pwlă",
    "p.u.l.a.",
    "poola",
    "naiba",
    "dracu",
    "draq",
    "drecu",
    "naibii",
    "dracului",
    "drecului",
    "drqlui",
    "coaie",
    "coae",
    "sloboz",
    "lindic",
    "gaoz",
    "ochiul maro",
    "floci",
    "cur",
    "futai",
    "futare",
    "futere",
    "popou",
    "nanau",
    "pulii",
    "pulii mele",
    "coaiele",
    "coaiele mele",
    "pulile",
    "măta",
    "mata",
    "mă-tii",
    "mă-ta",
    "mă-ti",
]

english_words = set(words.words())

def normalize_text(df):

    df['title'] = df['title'].apply(lambda x : re.sub(r"<.*?>", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"<.*?>", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"\.\.\.", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"\.\.\.", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"#[\w+-]+", '[HTAG]', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"#[\w+-]+", '[HTAG]', x))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[STAR]" if "*" in word else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x:' '.join(["[STAR]" if "*" in word else word for word in x.split()]))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))

    pattern = r'^[!?"\',“”-]*(.*?)[!?"\',“”-]*$'
    df['title'] = df['title'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))


    emoji_pattern = r"""
                    (?:
                      [<>]?
                      [:;=8]                     # eyes
                      [\-o\*\']?                 # optional nose
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      |
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      [\-o\*\']?                 # optional nose
                      [:;=8]                     # eyes
                      [<>]?
                      |
                      </?3                       # heart
                    )"""
    df['title'] = df['title'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))
    df['content'] = df['content'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))


    return df

In [27]:
def split_train_in_four(dataset):
    length = int(len(dataset) / 4)
    
    first = dataset[:length]
    second = dataset[length:2*length]
    third = dataset[2*length:3*length]
    fourth = dataset[3*length:4*length]

    first.to_csv('first.csv', encoding='utf-8', index=False)
    second.to_csv('second.csv', encoding='utf-8', index=False)
    third.to_csv('third.csv', encoding='utf-8', index=False)
    fourth.to_csv('fourth.csv', encoding='utf-8', index=False)
    
split_train_in_four(pd.read_csv('data/train.csv'))

In [54]:
def get_titles_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['title'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})
    
    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title, label in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)
        y_train.append(label)

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    return X_train, y_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['content'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})
    
    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))
        
    for content, label in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
            y_train.append(label)
        else:
            X_train.append([0 for _ in range(768)])
            y_train.append(label)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)
    
    text_embeddings, _ = get_titles_tokens(df)
    content_embeddings, _ = get_content_tokens(df)
    
    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)
    

In [57]:
df = pd.read_csv('fourth.csv')
preprocess_dataset(df, 'fourth')

100%|██████████| 17643/17643 [56:22<00:00,  5.22it/s]


In [ ]:
# AICI ESTE MODELUL PENTRU TITLU SI CONTENT

In [141]:
class TitleAndContent(nn.Module):
    def __init__(self):
        super(TitleAndContent, self).__init__()
        self.linear_text = nn.Linear(768, 128)
        self.linear_content = nn.Linear(768, 128)
        self.classifier = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, titles, contents):
        titles = self.linear_text(titles)
        contents = self.linear_content(contents)

        titles = self.relu(titles)
        contents = self.relu(contents)

        titles = self.dropout(titles)
        contents = self.dropout(contents)
        
        cat = torch.cat([titles, contents], axis = 1)
        
        out = self.classifier(cat)
        out = self.dropout(out)
        
        return self.sigmoid(out)

In [142]:
class TextAndContentDataset(Dataset):
    def __init__(self, x_titles, x_contents, y):
        super(TextAndContentDataset, self).__init__()
        self.x_titles = x_titles
        self.x_contents = x_contents
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, item):
        return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])

In [143]:
def get_training_data_content():
    first = pd.read_csv('first.csv')
    first_indices = first[first['content'].str.len() > 1].index.tolist()
    first_titles = np.load('training_splits/firsttitle.npy')
    first_titles = first_titles[first_indices]
    first_contents = np.load('training_splits/firstcontent.npy')
    first_contents = first_contents[first_indices]
    first_labels = np.array(first['class'].tolist())
    first_labels = first_labels[first_indices]

    second = pd.read_csv('second.csv')
    second_indices = second[second['content'].str.len() > 1].index.tolist()
    second_titles = np.load('training_splits/secondtitle.npy')
    second_titles = second_titles[second_indices]
    second_contents = np.load('training_splits/secondcontent.npy')
    second_contents = second_contents[second_indices]
    second_labels = np.array(second['class'].tolist())
    second_labels = second_labels[second_indices]

    third = pd.read_csv('third.csv')
    third_indices = third[third['content'].str.len() > 1].index.tolist()
    third_titles = np.load('training_splits/thirdtitle.npy')
    third_titles = third_titles[third_indices]
    third_contents = np.load('training_splits/thirdcontent.npy')
    third_contents = third_contents[third_indices]
    third_labels = np.array(third['class'].tolist())
    third_labels = third_labels[third_indices]

    fourth = pd.read_csv('fourth.csv')
    fourth_indices = fourth[fourth['content'].str.len() > 1].index.tolist()
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_titles = fourth_titles[fourth_indices]
    fourth_contens = np.load('training_splits/fourthcontent.npy')
    fourth_contens = fourth_contens[fourth_indices]
    fourth_labels = np.array(fourth['class'].tolist())
    fourth_labels = fourth_labels[fourth_indices]

    all_titles = np.concatenate((first_titles, second_titles, third_titles, fourth_titles), axis=0)
    all_contents = np.concatenate((first_contents, second_contents, third_contents, fourth_contens), axis=0)
    all_labels = np.concatenate((first_labels, second_labels, third_labels, fourth_labels))
    
    # all_titles = np.concatenate((first_titles, second_titles, third_titles), axis=0)
    # all_contents = np.concatenate((first_contents, second_contents, third_contents), axis=0)
    # all_labels = np.concatenate((first_labels, second_labels, third_labels))
    
    
    return all_titles, all_contents, all_labels

def get_validation_data_content():
    fourth = pd.read_csv('fourth.csv')
    fourth_indices = fourth[fourth['content'].str.len() > 1].index.tolist()
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_titles = fourth_titles[fourth_indices]
    fourth_contens = np.load('training_splits/fourthcontent.npy')
    fourth_contens = fourth_contens[fourth_indices]
    fourth_labels = np.array(fourth['class'].tolist())
    fourth_labels = fourth_labels[fourth_indices]

    return fourth_titles, fourth_contens, fourth_labels

In [195]:
X_train_titles, X_train_contents, y_train = get_training_data_content()
# X_val_titles, X_val_contents, y_val = get_validation_data_content()

In [206]:
train_set = TextAndContentDataset(X_train_titles, X_train_contents,y_train)
train_loader = DataLoader(train_set, shuffle=True, batch_size=256)

# val_set = TextAndContentDataset(X_val_titles, X_val_contents,y_val)
# val_loader = DataLoader(val_set, shuffle=True, batch_size=64)

In [207]:
model_text_and_content = TitleAndContent().to(device)
optimizer = optim.Adam(model_text_and_content.parameters(),lr=1e-4)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = nn.BCELoss(weight=torch.tensor([len(y_train) / np.sum(y_train)]).to(device))

In [208]:
EPOCHS = 100

In [209]:
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

for epoch in range(EPOCHS):
    running_loss = 0
    running_acc = 0

    train_pred_labels = []
    train_true_labels = []

    for X_title, X_content, y in tqdm(train_loader):
        y = y.to(device, dtype=torch.long)
        X_title = X_title.to(device, dtype=torch.float)
        X_content = X_content.to(device, dtype=torch.float)

        output = model_text_and_content(X_title, X_content)

        output.squeeze(1)
        pred = output > 0.5

        train_pred_labels.extend(pred.cpu().numpy().tolist())
        train_true_labels.extend(y.cpu().numpy().tolist())

        loss = criterion(output.squeeze(1), y.float())
        running_loss += loss.item()

        model_text_and_content.zero_grad()
        loss.backward()
        optimizer.step()

    pred_labels = []
    true_labels = []

    # model_text_and_content.eval()
    # val_loss = 0
    # with torch.no_grad():
    #     for X_title, X_content, y in tqdm(val_loader):
    #         y = y.to(device, dtype=torch.long)
    #         X_title = X_title.to(device, dtype=torch.float)
    #         X_content = X_content.to(device, dtype=torch.float)
    # 
    #         output = model_text_and_content(X_title, X_content)
    #         val_loss += criterion(output.squeeze(1), y.float()).item()
    #         output.squeeze(1)
    #         
    # 
    #         pred = (torch.sigmoid(output) > 0.5)
    #         pred_labels.extend(pred.cpu().numpy().tolist())
    #         true_labels.extend(y.cpu().numpy().tolist())
    #         
    #     # scheduler.step(val_loss)
            
    # 
    # model_text_and_content.train()
    # 
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
    precision_train, recall_train, f1_train, _ = precision_recall_fscore_support(train_pred_labels, train_true_labels, average='macro')


    # print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train} | Val Loss: {val_loss / len(val_set)} | Val Precision: {precision} | Val Recall: {recall} | Val F1: {f1}")
    print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train}")
    # print(f"Epoch: {epoch + 1} | Val Loss: {val_loss / len(val_set)} | Val Precision: {precision} | Val Recall: {recall} | Val F1: {f1}")
    
    # # save best model
    # if epoch == 0:
    #     best_f1 = f1
    #     torch.save(model_text_and_content.state_dict(), 'models/titleAndContentClassifier.pth')
    # else:
    #     if f1 > best_f1:
    #         best_f1 = f1
    #         torch.save(model_text_and_content.state_dict(), 'models/titleAndContentClassifier.pth')

torch.save(model_text_and_content.state_dict(), 'models/titleAndContentClassifier.pth')

  0%|          | 0/271 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\1227666216.py:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])
100%|██████████| 271/271 [00:01<00:00, 195.08it/s]


Epoch: 1 | Train Loss 0.0021509333851850027  | Train Precision: 0.8037687819272201 | Train Recall: 0.7834750045913559 | Train F1: 0.7863898192504182


  0%|          | 0/271 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\1227666216.py:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])
100%|██████████| 271/271 [00:01<00:00, 218.68it/s]


Epoch: 2 | Train Loss 0.0013768392061313333  | Train Precision: 0.8683203701908164 | Train Recall: 0.8413104390504298 | Train F1: 0.8415911202709866


  0%|          | 0/271 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\1227666216.py:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])
100%|██████████| 271/271 [00:01<00:00, 203.69it/s]


...

Epoch: 98 | Train Loss 0.0009102057526567018  | Train Precision: 0.896794508643279 | Train Recall: 0.8674573540512772 | Train F1: 0.8670115994814136


  0%|          | 0/271 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\1227666216.py:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])
100%|██████████| 271/271 [00:03<00:00, 87.36it/s]


Epoch: 99 | Train Loss 0.0009094126191911833  | Train Precision: 0.89727486078532 | Train Recall: 0.8679296170647639 | Train F1: 0.8676974396994261


  0%|          | 0/271 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_8184\1227666216.py:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.y[item])
100%|██████████| 271/271 [00:03<00:00, 81.24it/s]


Epoch: 100 | Train Loss 0.0009166747887697079  | Train Precision: 0.8969565419924246 | Train Recall: 0.8676185232185583 | Train F1: 0.8672615872464797


In [210]:
# load best model
model_text_and_content = TitleAndContent().to(device)
model_text_and_content.load_state_dict(torch.load('models/titleAndContentClassifier.pth'))

<All keys matched successfully>

In [211]:
# AICI SE TERMINA MODELUL PENTRU TITLU SI CONTENT
# DE AICI INCEPE PENTRU SUBMISIE

In [212]:
def get_all_test_data():
    file_names = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth']
    
    all_titles = np.array([])
    all_contents = np.array([])
    
    for f in file_names:
        titles = np.load(f'testing_splits/{f}_testtitle.npy')
        content = np.load(f'testing_splits/{f}_testcontent.npy')
        if len(all_titles) == 0:
            all_titles = titles
            all_contents = content
        else:
            all_titles = np.concatenate((all_titles, titles), axis=0)
            all_contents = np.concatenate((all_contents, content), axis=0)
            
    return all_titles, all_contents

In [213]:
X, y = get_all_test_data()
df = pd.read_csv('test.csv')

In [214]:
all_pred = []
ids = []

# model_titles = ClssificationNetwork().to(device)
# model_titles.load_state_dict(torch.load('haisavedem.pth'))

model_titles.eval()
model_text_and_content.eval()

indices = df[df['content'].str.len() > 1].index.tolist()

for i in tqdm(range(len(df))):
    title = X[i]
    content = y[i]
    if i in indices:
        title = torch.tensor(title, dtype=torch.float).to(device).unsqueeze(0)
        content = torch.tensor(content, dtype=torch.float).to(device).unsqueeze(0)
        output = model_text_and_content(title, content)
        output.squeeze(1)
        pred = (output > 0.5).item()
    else:
        title = torch.tensor(title, dtype=torch.float).to(device).unsqueeze(0)
        output = model_titles(title)
        output.squeeze(1)
        pred = (output > 0.5).item()
        
    all_pred.append(pred)
    ids.append(i)

100%|██████████| 36669/36669 [00:27<00:00, 1324.06it/s]


In [215]:
submisie = pd.DataFrame({'id':ids, 'class':list(map(int, all_pred))})
submisie.to_csv('submission_embeddings_100_epochs_fulltrain_batch256.csv', index=False)

In [ ]:
# GATAAAAAAAAAAAA

In [178]:
def get_titles_tokens(df):
    X_train = []
    aux = df['title'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)

    X_train = np.array(X_train)

    return X_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = df['content'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for content in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
        else:
            X_train.append([0 for _ in range(768)])

    X_train = np.array(X_train)

    return X_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)

    text_embeddings = get_titles_tokens(df)
    content_embeddings = get_content_tokens(df)

    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)

In [180]:
test_df = pd.read_csv('first_test.csv')
preprocess_dataset(test_df, 'first_test')

100%|██████████| 6111/6111 [16:26<00:00,  6.20it/s]


In [181]:
test_df = pd.read_csv('sixth_test.csv')
preprocess_dataset(test_df, 'sixth_test')

100%|██████████| 6114/6114 [22:24<00:00,  4.55it/s]
